--- Part Two ---
While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and record the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only 6 different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use O to mark the new obstruction, | to show a position where the guard moves up/down, - to show a position where the guard moves left/right, and + to show a position where the guard moves both up/down and left/right.

In [129]:
guard_characters = ["^", ">", "V", "<"]
pathing_chars = ["-", "|", "+"]
blocked_chars = [".", "#"]
initial_grid = [
    "....#.....",
    ".........#",
    "..........",
    "..#.......",
    ".......#..",
    "..........",
    ".#..^.....",
    "........#.",
    "#.........",
    "......#...",
]
initial_guard_loc_x = 0
initial_guard_loc_y = 0
initial_guard_direction = "^"

grid, guard_loc_x, guard_loc_y, guard_direction = (
    initial_grid,
    initial_guard_loc_x,
    initial_guard_loc_y,
    initial_guard_direction,
)

In [130]:
for y in range(len(grid)):
    for x in range(len(grid[y])):
        if grid[y][x] in guard_characters:
            guard_loc_x = x
            guard_loc_y = y
            guard_direction = grid[y][x]


def get_movement_dir(guard_dir):
    match guard_dir:
        case "^":
            return (0, -1, "|")
        case ">":
            return (1, 0, "-")
        case "V":
            return (0, 1, "|")
        case "<":
            return (-1, 0, "-")
        case _:
            return (0, 0, ".")


def get_lookahead(guard_loc_x, guard_loc_y, offset, grid, reference_char=["#"]):
    value = grid[guard_loc_y + offset[1]][guard_loc_x + offset[0]]
    return value in reference_char


def get_next_direction(guard_direction):
    index_of_direction = guard_characters.index(guard_direction)
    try:
        return guard_characters[index_of_direction + 1]
    except IndexError:
        return guard_characters[0]


def mutate_grid(x, y, grid, char):
    mutable_grid = [list(row) for row in grid]
    mutable_grid[y][x] = char
    grid = ["".join(row) for row in mutable_grid]
    return grid


def change_to_tracked(guard_loc_x, guard_loc_y, grid, direction_character):
    current_char = grid[guard_loc_y][guard_loc_x]
    if current_char in pathing_chars:
        direction_character = "+"
    grid = mutate_grid(guard_loc_x, guard_loc_y, grid, direction_character)
    return grid


def move_forward(guard_loc_x, guard_loc_y, guard_direction, grid, loop_reference):
    offset = (0, 0, ".")
    can_move_forward = False
    has_turned = False
    while can_move_forward == False:
        offset = get_movement_dir(guard_direction)
        can_move_forward = get_lookahead(
            guard_loc_x, guard_loc_y, offset, grid, ["#", "O"]
        )
        if can_move_forward == False:
            if get_lookahead(guard_loc_x, guard_loc_y, offset, grid, ["O"]) == False:
                if loop_reference == (guard_loc_y, guard_loc_x):
                    raise Exception("Closed loop")
                else:
                    loop_reference = (guard_loc_y, guard_loc_x)
            has_turned = True
            guard_direction = get_next_direction(guard_direction)
    grid = change_to_tracked(
        guard_loc_x, guard_loc_y, grid, "+" if has_turned else offset[2]
    )
    guard_loc_y = guard_loc_y + offset[1]
    guard_loc_x = guard_loc_x + offset[0]
    return (guard_loc_x, guard_loc_y, guard_direction, grid, loop_reference)

In [131]:
def run_full_analysis(guard_loc_x, guard_loc_y, guard_direction, grid):
    loop_reference = (-1, -1)
    
    while True:
        try:
            guard_loc_x, guard_loc_y, guard_direction, grid, loop_reference = (
                move_forward(
                    guard_loc_x, guard_loc_y, guard_direction, grid, loop_reference
                )
            )
        except IndexError:
            dir_char = get_movement_dir(guard_direction)
            grid = change_to_tracked(guard_loc_x, guard_loc_y, grid, dir_char[2])
            print("End of path")
            break
        except Exception as e:
            if e == "Closed loop":
                return 1
            break

    return 0



In [ ]:
end_count = 0
check_grid = initial_grid
end_count = run_full_analysis(guard_loc_x, guard_loc_y, guard_direction, check_grid)
for y in range(len(initial_grid)):
    for x in range(len(initial_grid[y])):
        grid, guard_loc_x, guard_loc_y, guard_direction = (
            initial_grid,
            initial_guard_loc_x,
            initial_guard_loc_y,
            initial_guard_direction,
        )
        grid = mutate_grid(x, y, grid, "O")
        for line in grid:
            print(line)
        end_count += run_full_analysis(guard_loc_x, guard_loc_y, guard_direction, grid)
print(end_count)

O...#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
-----
.O..#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
-----
..O.#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
-----
...O#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
-----
....O.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
-----
....#O....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
-----
....#.O...
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
-----
....#..O..
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
-----
....#...O.
.........#
..........
..#.......
.......#..
..........
.#..^.